In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import scipy
from scipy import stats
from scipy.stats import shapiro
from scipy.stats import mannwhitneyu

import warnings
warnings.filterwarnings('ignore')
warnings.warn('DelftStack')
warnings.warn('Do not show this message')

import statsmodels as sm
from statsmodels.stats.power import TTestIndPower

In [ ]:
df = pd.read_csv('cookie_cats.csv')
df.head(10)

,userid,version,sum_gamerounds,retention_1,retention_7
0,116,gate_30,3,0,0
1,337,gate_30,38,1,0
2,377,gate_40,165,1,0
3,483,gate_40,1,0,0
4,488,gate_40,179,1,1
5,540,gate_40,187,1,1
6,1066,gate_30,0,0,0
7,1444,gate_40,2,0,0
8,1574,gate_40,108,1,1
9,1587,gate_40,153,1,0


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90189 entries, 0 to 90188
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   userid          90189 non-null  int64 
 1   version         90189 non-null  object
 2   sum_gamerounds  90189 non-null  int64 
 3   retention_1     90189 non-null  int64 
 4   retention_7     90189 non-null  int64 
dtypes: int64(4), object(1)
memory usage: 3.4+ MB


In [ ]:
df.describe()

,userid,sum_gamerounds,retention_1,retention_7
count,9.018900e+04,90189.000000,90189.000000,90189.000000
mean,4.998412e+06,51.872457,0.445210,0.186065
std,2.883286e+06,195.050858,0.496992,0.389161
min,1.160000e+02,0.000000,0.000000,0.000000
25%,2.512230e+06,5.000000,0.000000,0.000000
50%,4.995815e+06,16.000000,0.000000,0.000000
75%,7.496452e+06,51.000000,1.000000,0.000000
max,9.999861e+06,49854.000000,1.000000,1.000000


Вывод: данные подгрузились корректно, значения цифровые, нет нулевых



In [ ]:
df.version.value_counts()

gate_40    45489
gate_30    44700
Name: version, dtype: int64

Распределение по группам примерно одинаковое

Далее проанализируем возвращаемость после 1 дня установки и после 7 дней установки

In [ ]:
control = df.query('version == "gate_30"')

test = df.query('version == "gate_40"')

Общая возвращаемость:

In [ ]:
df.retention_1.value_counts()

0    50036
1    40153
Name: retention_1, dtype: int64

In [ ]:
df.retention_1.value_counts()

0    50036
1    40153
Name: retention_1, dtype: int64

По группам:

In [ ]:
control.retention_1.value_counts()

0    24666
1    20034
Name: retention_1, dtype: int64

In [ ]:
test.retention_1.value_counts()

0    25370
1    20119
Name: retention_1, dtype: int64

In [ ]:
control.retention_7.value_counts()

0    36198
1     8502
Name: retention_7, dtype: int64

In [ ]:
test.retention_7.value_counts()

0    37210
1     8279
Name: retention_7, dtype: int64

Вывод: данные равномерно поделены по группам, резултаты возвращаемости в группах примерно одинаковые


Нулевая гипотеза - статитически значимой разницы между выборками нет\
alpha = 0.5

Проверим нашу гипотезу с помощью Критерий согласия Пирсона и z-теста

Для ретеншн = 1

In [17]:
n1, n2 = df.version.value_counts()

In [18]:
k1, k2 = df.query('retention_1 == 1').version.value_counts()

In [20]:
from statsmodels.stats import proportion

In [23]:
proportion.proportions_chisquare(np.array([k1, k2]), np.array([n1, n2]))

(3.182963657512031,
 0.07440965529692188,
 (array([[20119., 25370.],
         [20034., 24666.]]),
  array([[20252.13514952, 25236.86485048],
         [19900.86485048, 24799.13514952]])))

In [22]:
proportion.proportions_ztest(np.array([k1, k2]), np.array([n1, n2]))

(-1.7840862247974725, 0.07440965529691913)

Вывод:p-alpha > 0.07 - > стат различий для контрольной группы и тестовой нет


Проверим нормальность распределения

In [24]:
alpha = 0.05

st = shapiro(df.retention_1)
print('Discription is {}normal\n'.format( {True: 'not ',
False: ''}[st[1] < alpha]));

Discription is not normal



In [25]:
alpha = 0.05

st = shapiro(df.retention_7)
print('Discription is {}normal\n'.format( {True: 'not ',
False: ''}[st[1] < alpha]));

Discription is not normal



Используем критерий Манна-Уилкни чтобы сравнить влияет ли контрольная группа на количество сыгранных раундов

In [26]:
test_stat, pvalue = mannwhitneyu(df.loc[df["version"] == "gate_30", "sum_gamerounds"],
                            df.loc[df["version"] == "gate_40", "sum_gamerounds"])
print("Test Stat = %.4f, p-value = %.4f" % (test_stat, pvalue))

Test Stat = 1024331250.5000, p-value = 0.0502


Вывод:p-alpha > 0.05 - > стат различий для контрольной группы и тестовой нет

### Вывод: на основе полученных данных, можно сделать вывод о том, что для двух контрольных групп результаты по количеству сыгранных тестов и результаты по возвращаемосити в игру одинаковы и не зависят от выбора gate 30 или gate 40